In [43]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [80]:
train  = pd.read_csv('/Users/michael/Downloads/brokenbot_train.csv')
test = pd.read_csv('/Users/michael/Downloads/newhuman_train(1).csv', engine = 'python')
combi = train.append(test, ignore_index=True)
#print(test.shape)
combi.head()

,tweet,label
0,#ThingsDoneByMistake kissing auntie in the lips,1
1,RT @mc_derpin: #TheOlderWeGet the more pessimi...,1
2,RT @dmataconis: Ready To Feel Like A Failure? ...,1
3,Amen! #blacklivesmatter https://t.co/wGffaOqgzl,1
4,RT @NahBabyNah: Twitchy: Chuck Todd caught out...,1


In [81]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [83]:
# remove twitter handles (@user)
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")
# remove special characters, numbers, punctuations
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

combi.head()

,tweet,label,tidy_tweet
0,#ThingsDoneByMistake kissing auntie in the lips,1,#ThingsDoneByMistake kissing auntie lips
1,RT @mc_derpin: #TheOlderWeGet the more pessimi...,1,#TheOlderWeGet more pessimistic https jHZJl
2,RT @dmataconis: Ready To Feel Like A Failure? ...,1,Ready Feel Like Failure Joan Only When Burned ...
3,Amen! #blacklivesmatter https://t.co/wGffaOqgzl,1,Amen #blacklivesmatter https wGffaOqgzl
4,RT @NahBabyNah: Twitchy: Chuck Todd caught out...,1,Twitchy Chuck Todd caught there shilling Hilla...


In [84]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0        [#ThingsDoneByMistake, kissing, auntie, lips]
1    [#TheOlderWeGet, more, pessimistic, https, jHZJl]
2    [Ready, Feel, Like, Failure, Joan, Only, When,...
3         [Amen, #blacklivesmatter, https, wGffaOqgzl]
4    [Twitchy, Chuck, Todd, caught, there, shilling...
Name: tidy_tweet, dtype: object

In [85]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0              [#thingsdonebymistak, kiss, aunti, lip]
1       [#theolderweget, more, pessimist, http, jhzjl]
2    [readi, feel, like, failur, joan, onli, when, ...
3            [amen, #blacklivesmatt, http, wgffaoqgzl]
4    [twitchi, chuck, todd, caught, there, shill, h...
Name: tidy_tweet, dtype: object

In [86]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi['tidy_tweet'] = tokenized_tweet